In [ ]:
#!pip3 install --user --upgrade google-cloud-texttospeech

In [ ]:
#!pip install --upgrade google-cloud-speech

In [ ]:
#!pip install google-cloud-speech

In [17]:
import os

In [1]:
key_file = "dl-stfu-668ae575a084.json" ##you will need a key file.  we probs need some google credits for this.   talk to kilis

In [2]:
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file(key_file)


a useful link for the text to speach api in python
https://codelabs.developers.google.com/codelabs/cloud-text-speech-python3#6


In [3]:
import google.cloud.texttospeech as tts
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
import numpy as np
import random
import string
import pandas as pd

In [130]:
def unique_languages_from_voices(voices):
    language_set = set()
    for voice in voices:
        for language_code in voice.language_codes:
            language_set.add(language_code)
    return language_set


def list_languages():
    client = tts.TextToSpeechClient(credentials=credentials) ##need credentials in every client.  annoying
    response = client.list_voices()
    languages = unique_languages_from_voices(response.voices)

    return languages
        
def list_voices(language_code=None):
    client = tts.TextToSpeechClient(credentials=credentials) ##ditto
    response = client.list_voices(language_code=language_code)
    voices = sorted(response.voices, key=lambda voice: voice.name)

    
    return [(voice.name, tts.SsmlVoiceGender(voice.ssml_gender).name) for voice in  voices]
     
        
def text_to_wav(voice_name: str, text: str, speaking_rate = 1.0, pitch = 0.0):
    #speaking rate can be between 0.25 and 4.0.  outside that is err
    #pitch can be between -20.0 and 20.0. but don't mess that much.
    #https://cloud.google.com/python/docs/reference/texttospeech/latest/google.cloud.texttospeech_v1.types.AudioConfig
    #effects_profile_id worth explorign
    language_code = "-".join(voice_name.split("-")[:2])
    text_input = tts.SynthesisInput(text=text)
    voice_params = tts.VoiceSelectionParams(
        language_code=language_code, name=voice_name
    )
    
    #https://cloud.google.com/speech-to-text/docs/encoding
    audio_config = tts.AudioConfig(audio_encoding=tts.AudioEncoding.LINEAR16 
                                   ,speaking_rate=speaking_rate
                                   ,sampleRateHz=16000
                                   ,pitch=pitch)  ##this is where you get speed, pitch, etc.
    client = tts.TextToSpeechClient(credentials=credentials)
    response = client.synthesize_speech(
        input=text_input, voice=voice_params, audio_config=audio_config
    )

    filename = f"{language_code}.wav"
    with open(filename, "wb") as out:
        out.write(response.audio_content)
        print(f'Generated speech saved to "{filename}"')
        
def random_settings_text_to_wav( text: str, data_path: str, voice_list, profile_list):
    """
    data_path should be the parent folder. in data_path you should have one folder for each output classes.  
    """
    voice = random.choice(voice_list) #takes the list as i prepare it in list_voices
    voice_name = voice[0]
    gender = voice[1]
    
    speaking_rate = random.uniform(0.8,1.2) #do i need to reduce digits?
    pitch = random.uniform(-5,5)
    profile = random.choice(profile_list)
    print(gender, profile)
    #speaking rate can be between 0.25 and 4.0.  outside that is err
    #pitch can be between -20.0 and 20.0. but don't mess that much.
    #https://cloud.google.com/python/docs/reference/texttospeech/latest/google.cloud.texttospeech_v1.types.AudioConfig
    #effects_profile_id worth explorign
    
    language_code = "-".join(voice_name.split("-")[:2])
    text_input = tts.SynthesisInput(text=text)
    voice_params = tts.VoiceSelectionParams(
        language_code=language_code, name=voice_name, ssml_gender = gender
    )
    audio_config = tts.AudioConfig(audio_encoding=tts.AudioEncoding.LINEAR16
                                   ,speaking_rate=speaking_rate
                                   ,sample_rate_hertz=16_000
                                   ,pitch=pitch
                                  , effects_profile_id = [profile] #this appears to be causing errors.
                                  )  ##this is where you get speed, pitch, etc.
    client = tts.TextToSpeechClient(credentials=credentials)
    response = client.synthesize_speech(
        input=text_input, voice=voice_params, audio_config=audio_config
    )
    code = ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(15)) #file name vishesh no repeats
    filename = f"{language_code}_{code}.wav"
    #this needs to be written to a file somewhere. the text, the file name, and the spectrogram too.  
    #i need ghz i think
    
    #text should match the folder.  if it doesn't exist create it?
    data_contents = os.listdir(data_path)
    output_dir = [i for i in contents if i[0] != '.']
    
    if text not in class_dir:
        raise ValueError('you must have a folder that matches the text')
    
    save_path = os.path.join(data_path,text)
    file_path = os.path.join(save_path,filename)
    print(file_path)
    #raise ValueError('stop!')
    with open(file_path, "wb") as out:
        out.write(response.audio_content)
        print(f'Generated speech saved to "{file_path}"')

In [131]:
voice_list = list_voices(language_code="en-us")
voice_list

[('en-US-Neural2-A', 'MALE'),
 ('en-US-Neural2-C', 'FEMALE'),
 ('en-US-Neural2-D', 'MALE'),
 ('en-US-Neural2-E', 'FEMALE'),
 ('en-US-Neural2-F', 'FEMALE'),
 ('en-US-Neural2-G', 'FEMALE'),
 ('en-US-Neural2-H', 'FEMALE'),
 ('en-US-Neural2-I', 'MALE'),
 ('en-US-Neural2-J', 'MALE'),
 ('en-US-News-K', 'FEMALE'),
 ('en-US-News-L', 'FEMALE'),
 ('en-US-News-M', 'MALE'),
 ('en-US-News-N', 'MALE'),
 ('en-US-Polyglot-1', 'MALE'),
 ('en-US-Standard-A', 'MALE'),
 ('en-US-Standard-A', 'MALE'),
 ('en-US-Standard-B', 'MALE'),
 ('en-US-Standard-B', 'MALE'),
 ('en-US-Standard-C', 'FEMALE'),
 ('en-US-Standard-C', 'FEMALE'),
 ('en-US-Standard-D', 'MALE'),
 ('en-US-Standard-D', 'MALE'),
 ('en-US-Standard-E', 'FEMALE'),
 ('en-US-Standard-E', 'FEMALE'),
 ('en-US-Standard-F', 'FEMALE'),
 ('en-US-Standard-F', 'FEMALE'),
 ('en-US-Standard-G', 'FEMALE'),
 ('en-US-Standard-G', 'FEMALE'),
 ('en-US-Standard-H', 'FEMALE'),
 ('en-US-Standard-H', 'FEMALE'),
 ('en-US-Standard-I', 'MALE'),
 ('en-US-Standard-I', 'MALE'),

In [132]:
profile_id_list = ['wearable-class-device'##	Smart watches and other wearables, like Apple Watch, Wear OS watch
,'handset-class-device'##	Smartphones, like Google Pixel, Samsung Galaxy, Apple iPhone
,'headphone-class-device'##	Earbuds or headphones for audio playback, like Sennheiser headphones
,'small-bluetooth-speaker-class-device'#	Small home speakers, like Google Home Mini
,'medium-bluetooth-speaker-class-device'#	Smart home speakers, like Google Home
,'large-home-entertainment-class-device'#	Home entertainment systems or smart TVs, like Google Home Max, LG TV
,'large-automotive-class-device'#	Car speakers
,'telephony-class-application'#	Interactive Voice Response (IVR) systems
]

In [133]:
words = ['up'
         ,'down'
         ,'left'
         ,'right'
         ,'yes'
         ,'no'
         ,'on'
         ,'off'
         ,'stop'
         ,'go']


In [134]:
word = random.choice(words)
word = "off"
random_settings_text_to_wav( word, '/Users/dallin.gordon/Desktop/s4_files/data_aug/data/', voice_list,profile_id_list) #fix the save, where we want it vish?

FEMALE small-bluetooth-speaker-class-device
/Users/dallin.gordon/Desktop/s4_files/data_aug/data/off/en-US_YVGE8PGZVHEOW8N.wav
Generated speech saved to "/Users/dallin.gordon/Desktop/s4_files/data_aug/data/off/en-US_YVGE8PGZVHEOW8N.wav"


In [79]:
import IPython

IPython.display.Audio("/Users/dallin.gordon/Desktop/s4_files/data_aug/data/off/en-US_8BXMZDYL2MZDY4Q.wav")

In [44]:
directory_path = '/Users/dallin.gordon/Desktop/s4_files/data_aug/data/'

# Use os.listdir to get a list of all files and directories in the specified directory
contents = os.listdir(directory_path)
class_dir = [i for i in contents if i[0] != '.']

In [45]:
class_dir

['on', 'off']

In [135]:
import numpy as np
from scipy.io import wavfile

# Load the wave file
x = wavfile.read("/Users/dallin.gordon/Desktop/s4_files/data_aug/data/off/en-US_YVGE8PGZVHEOW8N.wav")

In [136]:
x

(16000, array([0, 0, 0, ..., 0, 0, 0], dtype=int16))

In [129]:
dir(tts.AudioConfig())

['__bool__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'audio_encoding',
 'effects_profile_id',
 'pitch',
 'sample_rate_hertz',
 'speaking_rate',
 'volume_gain_db']